# Building prediction models using the created embeddings

This notebook is a step-by-step guide to create multiple prediction models with the created embeddings. The code can only be runned after creating the embedding by using the GLACE train.py file.

The notebook consist out of three main sections.
- The first sections combines the chosen embedding together with the raw data after which it constructs three different models: Linear regression, XGBoost regressor and Random Forest regressor. For each of the models the code to perform the gridsearch is also present. 
- The second section utilizes only the raw data features and was used to acquire the baseline results.
- The third section contains the feature importance analysis using Shap-values

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit, train_test_split
import sklearn.metrics
from sklearn.preprocessing import normalize
from sklearn.linear_model import LinearRegression, ElasticNet
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns
from scipy.stats import norm, skew 
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Modelling for embedding + RAW

### Read in data and drop variables that skew the performance

In [ ]:
# Network embedding generated by GLACE
ps = pd.read_pickle('GLACE/emb/glace_cora_ml_embedding_first-order_Final_house.pkl')
labels = pd.read_csv("price_label_final.csv")
base_df = pd.read_csv("BaseKCFinal.csv")
base_df = base_df.drop(["id"], axis=1)
base_df = base_df.drop(["index"], axis=1)
base_df = base_df.drop(base_df.columns[0],axis = 1)
base_df.head

In [ ]:
# Example of the content of the network embedding for node 2.
print("mu of node 2: ", ps["mu"][2])
print(100*"*")
print("sigma of node 2: ", ps["sigma"][2])

Run following code only if influence of latitude and longitude on results has to be tested

In [ ]:
#Create raw set without lat and lon
df_no_lat =  base_df.drop(['lat','long'], axis = 1)
base_df = df_no_lat

## Performing train-test split

**Run if House-House embedding**

In [ ]:
test_size = round(len(labels)*0.2)
X_train =  np.array([np.array(ps['mu'][k]) for k in range(0, len(ps['mu']) - test_size)])
X_test = np.array([np.array(ps['mu'][k]) for k in range(len(ps['mu']) - test_size, len(ps['mu']))])

y_train =  np.array([np.array(labels['price'][k]) for k in range(0, len(labels['price']) - test_size)])
y_test = np.array([np.array(labels['price'][k]) for k in range(len(labels['price']) - test_size, len(labels['price']))])

**Run if House-School embedding**

Different code since only the information of the houses will be used. The embedding variables already contain the information of nearby schools

In [ ]:
test_split = len(labels) - round(len(labels)*0.2)
X_train =  np.array([np.array(ps['mu'][k]) for k in range(0,test_split)])
X_test = np.array([np.array(ps['mu'][k]) for k in range(test_split, len(labels))])

y_train =  np.array([np.array(labels['price'][k]) for k in range(0,test_split)])
y_test = np.array([np.array(labels['price'][k]) for k in  range(test_split, len(labels))])

Convert 2D numpy array into pandas dataframes

In [ ]:
X_train = pd.DataFrame(X_train.tolist(), columns=["col_" + str(i) for i in range(X_train.shape[1])])
X_test = pd.DataFrame(X_test.tolist(), columns=["col_" + str(i) for i in range(X_test.shape[1])])

Check dataframe shapes. Number of columns should be the same!


In [ ]:
print("train shape: ", X_train.shape)
print("test shape: ", X_test.shape)
print("base_Df shape: ", base_df.shape)

Reshape raw data to make sure that indices line up with their respective nodes. Adjust numbers based on the shape sizes of previous code. 

14270 houses are part of the train set and 3568 houses are part of the test set. House instances were ordered on their index and train-test split hasn't been performed randomly. This has been done to ensure we can add the correct raw data to the correct houses.

In [ ]:
base_df_train = base_df.head(14270)
base_df_train.shape
base_df_test = base_df.tail(3568)
base_df_test.reset_index(drop=True, inplace=True)
base_df_test.shape

Concatenate the dataframe containing the values of the network embedding and the data from the raw dataframe. Also remove label from the dataset.

In [ ]:
X_train_concatenated = pd.concat([X_train, base_df_train], axis=1)
X_test_concatenated = pd.concat([X_test, base_df_test], axis=1)
X_train_concatenated.drop(["price"], inplace=True, axis=1)
X_test_concatenated.drop(["price"], inplace=True, axis=1)
X_train = X_train_concatenated
X_test = X_test_concatenated


## Preprocessing & exploration

### Exploration of the target variable (price) distribution

Investigate the distribution of the target variable. 

In [ ]:
sns.distplot(y_train , fit=norm)

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(y_train)
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

# Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

# Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(y_train, plot=plt)
plt.show()

The distribution of the target variable is skewed. This could affect the performance of the prediction models

Perform log-transformation on the target variable in the training and test set.

In [ ]:
#We use the numpy fuction log1p which  applies log(1+x) to all elements of the column
y_train = np.log1p(y_train)

#Check the new distribution 
sns.distplot(y_train , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(y_train)
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(y_train, plot=plt)
plt.show()

In [ ]:
# We use the numpy fuction log1p which applies log(1+x) to all elements of the column
y_test = np.log1p(y_test)

#Check the new distribution 
sns.distplot(y_test , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(y_test)
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

# Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

# Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(y_test, plot=plt)
plt.show()

## Prediction models for embedding + raw data

Define MAPE metric, this will be used in addition to MAE and RMSE to evaluate the models

In [ ]:
def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

### Linear regression

No Gridsearch preformed for linear regression due to being irrelevant. Ridge, Lasso and ElasticNet models have been tested but performed worse compared to linear regression, thus the code was removed.

In [ ]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)

y_pred = linreg.predict(X_test)
#Test set results
rmse = mean_squared_error(y_test, y_pred, squared=False)
rmselog = mean_squared_error(np.expm1(y_test), np.expm1(y_pred), squared=False)
mae = mean_absolute_error(y_test, y_pred)
maelog = mean_absolute_error(np.expm1(y_test), np.expm1(y_pred))
mape = MAPE(y_test,y_pred)
mapelog = MAPE(np.expm1(y_test), np.expm1(y_pred))
#Training set results
y_pred_tr = linreg.predict(X_train)
rmse_tr = mean_squared_error(y_train, y_pred_tr, squared=False)
mae_tr = mean_absolute_error(y_train, y_pred_tr)
mape_tr = MAPE(y_train, y_pred_tr)
print('Results of test set')
print('R-squared|', linreg.score(X_test, y_test))
print('RMSE_test|', round(rmse,4))
print('MAE_test |', round(mae,4))
print('MAPE_test|', round(mape,2), '%')
print('*'*100)
print('Results of training set')
print('R-squared|', linreg.score(X_train, y_train))
print('RMSE_train|', round(rmse_tr,4))
print('MAE_train |', round(mae_tr,4))
print('MAPE_train|', round(mape_tr,2), '%')
print('*'*100)
print('RMSE_testLOG|', round(rmselog,4))
print('MAE_testLOG|', round(maelog,4))
print('MAPE_testLOG|', round(mapelog,2), '%')

### XGBoost regressor

Gridsearch for XGB Regressor. Not all tested parameters being present due to not being able to run the code in one go. 

In [ ]:

XGBR = XGBRegressor(objective = 'reg:squarederror')
grid_param = {'n_estimators': [750,1000],
              'max_depth': [7,9],
              'learning_rate' : [0.02,0.03]
              }
grid_mse = GridSearchCV(estimator = XGBR, param_grid = grid_param, scoring = "neg_mean_squared_error", cv = 5, verbose = 1)
grid_mse.fit(X_train, y_train,verbose=True)
print("Best parameters found: ", grid_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid_mse.best_score_)))

In [ ]:
XGBR = XGBRegressor(n_estimators = 750, objective = 'reg:squarederror', max_depth = 7, learning_rate = 0.02)
XGBR.fit(X_train, y_train,verbose=True)
y_pred = XGBR.predict(X_test)
#Test set results
rmse = mean_squared_error(y_test, y_pred, squared=False)
rmselog = mean_squared_error(np.expm1(y_test), np.expm1(y_pred), squared=False)
mae = mean_absolute_error(y_test, y_pred)
maelog = mean_absolute_error(np.expm1(y_test), np.expm1(y_pred))
mape = MAPE(y_test,y_pred)
mapelog = MAPE(np.expm1(y_test), np.expm1(y_pred))
#Training set results
y_pred_tr = XGBR.predict(X_train)
rmse_tr = mean_squared_error(y_train, y_pred_tr, squared=False)
mae_tr = mean_absolute_error(y_train, y_pred_tr)
mape_tr = MAPE(y_train, y_pred_tr)
print('Results of test set')
print('R-squared|', XGBR.score(X_test, y_test))
print('RMSE_test|', round(rmse,4))
print('MAE_test |', round(mae,4))
print('MAPE_test|', round(mape,2), '%')
print('*'*100)
print('Results of training set')
print('R-squared|', XGBR.score(X_train, y_train))
print('RMSE_train|', round(rmse_tr,4))
print('MAE_train |', round(mae_tr,4))
print('MAPE_train|', round(mape_tr,2), '%')
print('*'*100)
print('RMSE_testLOG|', round(rmselog,4))
print('MAE_testLOG|', round(maelog,4))
print('MAPE_testLOG|', round(mapelog,2), '%')

### Random Forest regressor

The random forest model takes the longest to run

Gridsearch for Random Forest regressor

In [ ]:
rf = RandomForestRegressor()
grid_param = {'n_estimators': [500,800],
              'max_depth': [11,13,15],
              'min_samples_split': [2,5],
              }
grid_mse = GridSearchCV(estimator = rf, param_grid = grid_param, scoring = "neg_mean_squared_error", cv = 5, verbose = 1)
grid_mse.fit(X_train, y_train)
print("Best parameters found: ", grid_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid_mse.best_score_)))

In [ ]:
rf = RandomForestRegressor(n_estimators=800, max_depth=15, min_samples_split=5)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
#Test set results
rmse = mean_squared_error(y_test, y_pred, squared=False)
rmselog = mean_squared_error(np.expm1(y_test), np.expm1(y_pred), squared=False)
mae = mean_absolute_error(y_test, y_pred)
maelog = mean_absolute_error(np.expm1(y_test), np.expm1(y_pred))
mape = MAPE(y_test,y_pred)
mapelog = MAPE(np.expm1(y_test), np.expm1(y_pred))
#Training set results
y_pred_tr = rf.predict(X_train)
rmse_tr = mean_squared_error(y_train, y_pred_tr, squared=False)
mae_tr = mean_absolute_error(y_train, y_pred_tr)
mape_tr = MAPE(y_train, y_pred_tr)
print('Results of test set')
print('R-squared|', rf.score(X_test, y_test))
print('RMSE_test|', round(rmse,4))
print('MAE_test |', round(mae,4))
print('MAPE_test|', round(mape,2), '%')
print('*'*100)
print('Results of training set')
print('R-squared|',rf.score(X_train, y_train))
print('RMSE_train|', round(rmse_tr,4))
print('MAE_train |', round(mae_tr,4))
print('MAPE_train|', round(mape_tr,2), '%')
print('*'*100)
print('RMSE_testLOG|', round(rmselog,4))
print('MAE_testLOG|', round(maelog,4))
print('MAPE_testLOG|', round(mapelog,2), '%')

## Modelling for RAW-only 

Feature engineering is not included with regards to geospatial information like amount of houses in certain proximity due to time limitations.

### Read in data and drop variables that skew the performance

In [ ]:
base_df = pd.read_csv("BaseKCFinal.csv")
base_df = base_df.drop(["id"], axis=1)
base_df = base_df.drop(["index"], axis=1)
base_df = base_df.drop(base_df.columns[0],axis = 1)
y = base_df["price"]
X = base_df.drop(["price"], axis=1)

Run following code only if influence of latitude and longitude on results has to be tested

In [ ]:
#Raw set without lat and lon
df_no_lat =  base_df.drop(['lat','long'], axis = 1)
base_df = df_no_lat

Perform train-test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=420)

Transforming house prices


In [ ]:
y_train = np.log1p(y_train)
y_test = np.log1p(y_test)

### Linear Regression 


In [ ]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_test)
#Test set results
rmse = mean_squared_error(y_test, y_pred, squared=False)
rmselog = mean_squared_error(np.expm1(y_test), np.expm1(y_pred), squared=False)
mae = mean_absolute_error(y_test, y_pred)
maelog = mean_absolute_error(np.expm1(y_test), np.expm1(y_pred))
mape = MAPE(y_test,y_pred)
mapelog = MAPE(np.expm1(y_test), np.expm1(y_pred))
#Training set results
y_pred_tr = linreg.predict(X_train)
rmse_tr = mean_squared_error(y_train, y_pred_tr, squared=False)
mae_tr = mean_absolute_error(y_train, y_pred_tr)
mape_tr = MAPE(y_train, y_pred_tr)
print('Results of test set')
print('R-squared|', linreg.score(X_test, y_test))
print('RMSE_test|', round(rmse,4))
print('MAE_test |', round(mae,4))
print('MAPE_test|', round(mape,2), '%')
print('*'*100)
print('Results of training set')
print('R-squared|', linreg.score(X_train, y_train))
print('RMSE_train|', round(rmse_tr,4))
print('MAE_train |', round(mae_tr,4))
print('MAPE_train|', round(mape_tr,2), '%')
print('*'*100)
print('RMSE_testLOG|', round(rmselog,4))
print('MAE_testLOG|', round(maelog,4))
print('MAPE_testLOG|', round(mapelog,2), '%')

### XGBoost regressor

Gridsearch for RAW data

In [ ]:
XGBR = XGBRegressor(objective = 'reg:squarederror')
grid_param = {'n_estimators': [1000,1500],
              'max_depth': [5,7],
              'learning_rate' : [0.02,0.025]
              }
grid_mse = GridSearchCV(estimator = XGBR, param_grid = grid_param, scoring = "neg_mean_squared_error", cv = 5, verbose = 1)
grid_mse.fit(X_train, y_train,verbose=True)
print("Best parameters found: ", grid_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid_mse.best_score_)))

In [ ]:
XGBR = XGBRegressor(n_estimators = 1000, objective = 'reg:squarederror', max_depth = 7, learning_rate = 0.025)
XGBR.fit(X_train, y_train)
y_pred = XGBR.predict(X_test)
#Test set results
rmse = mean_squared_error(y_test, y_pred, squared=False)
rmselog = mean_squared_error(np.expm1(y_test), np.expm1(y_pred), squared=False)
mae = mean_absolute_error(y_test, y_pred)
maelog = mean_absolute_error(np.expm1(y_test), np.expm1(y_pred))
mape = MAPE(y_test,y_pred)
mapelog = MAPE(np.expm1(y_test), np.expm1(y_pred))
#Training set results
y_pred_tr = XGBR.predict(X_train)
rmse_tr = mean_squared_error(y_train, y_pred_tr, squared=False)
mae_tr = mean_absolute_error(y_train, y_pred_tr)
mape_tr = MAPE(y_train, y_pred_tr)
print('Results of test set')
print('R-squared|', XGBR.score(X_test, y_test))
print('RMSE_test|', round(rmse,4))
print('MAE_test |', round(mae,4))
print('MAPE_test|', round(mape,2), '%')
print('*'*100)
print('Results of training set')
print('R-squared|', XGBR.score(X_train, y_train))
print('RMSE_train|', round(rmse_tr,4))
print('MAE_train |', round(mae_tr,4))
print('MAPE_train|', round(mape_tr,2), '%')
print('*'*100)
print('RMSE_testLOG|', round(rmselog,4))
print('MAE_testLOG|', round(maelog,4))
print('MAPE_testLOG|', round(mapelog,2), '%')

### Random Forest regressor

Gridsearch for RAW data

In [ ]:
rf = RandomForestRegressor()
grid_param = {'n_estimators': [500,750],
              'max_depth': [11],
              'min_samples_split': [2,5],
              }
grid_mse = GridSearchCV(estimator = rf, param_grid = grid_param, scoring = "neg_mean_squared_error", cv = 5, verbose = 1)
grid_mse.fit(X_train, y_train)
print("Best parameters found: ", grid_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid_mse.best_score_)))

In [ ]:
rf = RandomForestRegressor(n_estimators=500, max_depth=11,min_samples_split=2)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
#Test set results
rmse = mean_squared_error(y_test, y_pred, squared=False)
rmselog = mean_squared_error(np.expm1(y_test), np.expm1(y_pred), squared=False)
mae = mean_absolute_error(y_test, y_pred)
maelog = mean_absolute_error(np.expm1(y_test), np.expm1(y_pred))
mape = MAPE(y_test,y_pred)
mapelog = MAPE(np.expm1(y_test), np.expm1(y_pred))
#Training set results
y_pred_tr = rf.predict(X_train)
rmse_tr = mean_squared_error(y_train, y_pred_tr, squared=False)
mae_tr = mean_absolute_error(y_train, y_pred_tr)
mape_tr = MAPE(y_train, y_pred_tr)
print('Results of test set')
print('R-squared|', rf.score(X_test, y_test))
print('RMSE_test|', round(rmse,4))
print('MAE_test |', round(mae,4))
print('MAPE_test|', round(mape,2), '%')
print('*'*100)
print('Results of training set')
print('R-squared|',rf.score(X_train, y_train))
print('RMSE_train|', round(rmse_tr,4))
print('MAE_train |', round(mae_tr,4))
print('MAPE_train|', round(mape_tr,2), '%')
print('*'*100)
print('RMSE_testLOG|', round(rmselog,4))
print('MAE_testLOG|', round(maelog,4))
print('MAPE_testLOG|', round(mapelog,2), '%')

# Feature Importance testing using SHAP values
Utilize the test set if you want to better understand the predictions of the models on unseen data, the train set if 
you want to investigate the behavior of the model on specific training instances.

1) XGBRegressor
2) RandomForestRegressor


In [ ]:
# import pickle
# with open('XGB_Final', 'wb') as file:
#         pickle.dump(XGBR, file)

Initially the shapley values must be computed of the various features.

In [ ]:
import shap
def construct_shap_values(model):
    explainer = shap.Explainer(model)
    instance_index = 0
    feature_names = X_test_concatenated.columns
    shap_values = explainer(X_test_concatenated)
    shap_values_df = pd.DataFrame(shap_values.values[0], index=feature_names, columns=["Shapley Values"])
    pd.set_option("display.max_rows", None)
    print(shap_values_df)
    return shap_values_df, shap_values
shap_values_df, shap_values = construct_shap_values(XGBR)
 # Onderstaande output moet in text editor geopend worden, anders ziet ge de volledige output niet. 

The beeswarm plot visualizes the SHAP values per instance, which gives an insight on how the features are contributing to the model's predictions.
Each dot in the plot bellow represents an instance of the test dataset, its position on the x-axis indicates the SHAP value for that instance and feature. A positive SHAP value means that the feature increases the prediction value relative to the average prediction of the model, negative SHAP value decreases the prediction value.

The more spread out the dots are the more varied the impact of the feature is on the predictions. More dots clustered around the positive side means that a feature generally increases the prediction value, vice versa for the negative side. 

The color scheme on the side indicates the relationship between the feature values and their impact on the prediction of the model. In other words, it indicates the significance of the feature in influencing the outcome of the prediction model. 

E.g. High feature values with consistently high SHAP values, means that higher values of that feature lead to an increase in prediction. 
E.g. King county data: "lat" has more often a high feature importance, which implies that the variable has a significant impact on determining the prediction of the house price. It however also has a wide distribution, this indicates that the variable may be nonlinear or depend on other features in the model, such as "lon". 

In [ ]:
def visualize_beeswarm(subset_shap_values):
    total_features = X_test_concatenated.shape[1]
    shap.plots.beeswarm(subset_shap_values, max_display=15)

instance_indices = X_test_concatenated.index.to_numpy()
subset_shap_values = shap_values[instance_indices]
#subset_shap_values = subset_shap_values[:,:10]
visualize_beeswarm(subset_shap_values)
subset_shap_values

In [ ]:
shap.summary_plot(shap_values, X_test, feature_names=X_test_concatenated.columns, plot_type='violin', show=False)

plt.tight_layout()
plt.subplots_adjust(left=0.3)
plt.xlabel("SHAP Value")
plt.ylabel("Feature")


plt.show()

The bar chart indicates the average impact of the various features.

In [ ]:
shap.plots.bar(shap_values)

The code below does the same but for the random forest model.

In [ ]:
shap_values_df_rf, shap_values_rf = construct_shap_values(rf)

In [ ]:
visualize_beeswarm(shap_values_rf)

In [ ]:
shap.plots.bar(shap_values_df_rf)